In [6]:
import torch
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

ModuleNotFoundError: No module named 'tensorflow'

In [2]:
device = "cpu"
if torch.cuda.is_available():
    # Get the number of available GPUs
    num_gpus = torch.cuda.device_count()
    print("CUDA is available with", num_gpus, "GPU(s)!")
    device = "cuda"
    # Print information about each GPU
    for i in range(num_gpus):
        print("GPU", i, ":", torch.cuda.get_device_name(i))
else:
    print("CUDA is not available. Running on CPU.")

print(device)

CUDA is available with 1 GPU(s)!
GPU 0 : NVIDIA GeForce RTX 4070
cuda
